### **Balanced Risk Set Matching in Observational Studies**
---
**Authors:**  
📌 *Justin Chris Catingub*  
📌 *Juliana Marie Ochea*  
---
    
## **Data Analytics Assignment**

### **Introduction**

In observational studies, treatment is often not randomly assigned, making causal inference difficult.
To reduce bias, we use Balanced Risk Set Matching (BRSM), a technique that pairs treated individuals
with similar controls.

This notebook implements BRSM using Mahalanobis Distance Matching, based on the journal 'Balanced Risk Set Matching'.


# **Step 1: Import Necessary Libraries**